In [1]:
import whisper
import subprocess as sp
from pathlib import Path
import json
import pandas as pd

RECORDINGS = Path('../server/trpl-recordings/')

In [2]:
model = whisper.load_model("base")

In [3]:
answers = [json.loads(l) for l in open('../server/trpl-evaluator.txt', 'r').readlines()]    

In [4]:
ans = answers[-1]

for p in RECORDINGS.iterdir():
    if p.stem.endswith(ans['uuid']):
        video_path = p    

In [5]:
result = model.transcribe(str(video_path))
text = result['text']

/home/will/ownership-inventory/notebooks/.env/lib/python3.7/site-packages/whisper/transcribe.py:78: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


In [16]:
with open('words.txt', 'w') as f:
    f.write(text)

aligned = sp.check_output(
    f'curl -F "audio=@{video_path}" -F "transcript=@words.txt" "http://localhost:32768/transcriptions?async=false"',
    shell=True).decode('utf-8')

aligned = json.loads(aligned)['words']

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1453k    0 18395  100 1435k   1124  89878  0:00:16  0:00:16 --:--:--     0


In [8]:
df = pd.DataFrame(ans['answers'])
df['start'] -= ans['start']
df['end'] -= ans['start']
df['start'] /= 1000.
df['end'] /= 1000.

,answer,correct,start,end
0,"{'doesCompile': False, 'lineNumber': 2}",False,5.384,13.983
1,"{'doesCompile': True, 'stdout': 'hello '}",False,13.984,26.188


In [65]:
from intervaltree import IntervalTree

cur_time = 0
cur_idx = 0

segments = IntervalTree()
for entry in aligned:
    word = entry['word']
    word_time_start = entry['start']
    word_time_end = entry['end']    
    word_idx_start = text[cur_idx:].find(word) + cur_idx
    word_idx_end = word_idx_start + len(word)
    
    if abs(cur_time - word_time_start) > 0.001:
        segments[cur_time:word_time_start] = [cur_idx, word_idx_start]
        
    segments[word_time_start:word_time_end] = [word_idx_start, word_idx_end]
    
    cur_idx = word_idx_end
    cur_time = word_time_end

In [77]:
for _, row in df.iterrows():
    segs = sorted(segments[row.start:row.end])
    ans_text = text[segs[0].data[0]:segs[-1].data[1]]